# Введение #

В этом уроке мы посмотрим, как строить нейронные сети, способные изучать сложные зависимости, которыми славятся глубокие нейронные сети.

Ключевая идея здесь — *модульность*: построение сложной сети из более простых функциональных единиц. Мы видели, как линейный элемент вычисляет линейную функцию — теперь посмотрим, как комбинировать и модифицировать такие единицы, чтобы моделировать более сложные зависимости.

# Слои #

Обычно нейронные сети организуют нейроны в **слои**. Когда мы объединяем линейные элементы с общим набором входов, получаем **dense**-слой.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/2MA4iMV.png" width="300" alt="A stack of three circles in an input layer connected to two circles in a dense layer.">
<figcaption style="textalign: center; font-style: italic"><center>Dense-слой из двух линейных элементов, получающих два входа и смещение.
</center></figcaption>
</figure>

Можно думать о каждом слое в нейронной сети как о некотором относительно простом преобразовании. Через глубокий стек слоёв нейронная сеть может преобразовывать свои входы всё более сложными способами. В хорошо обученной нейронной сети каждый слой — это преобразование, которое приближает нас на шаг к решению.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
    <strong>Многие типы слоёв</strong><br>
«Слой» в Keras — очень общее понятие. Слой может быть, по сути, любым видом <em>преобразования данных</em>. Многие слои, такие как <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D">свёрточные</a> и <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN">рекуррентные</a>, преобразуют данные с помощью нейронов и отличаются главным образом схемой связей. Другие используются для <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding">инженерии признаков</a> или просто <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add">простых арифметических операций</a>. Здесь целый мир слоёв — <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">посмотрите</a>!
</blockquote>

# Функция активации #

Однако оказывается, что два dense-слоя подряд без ничего между ними не лучше одного dense-слоя. Сами по себе dense-слои никогда не выводят нас за пределы мира линий и плоскостей. Нам нужно что-то *нелинейное*. Нам нужны функции активации.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/OLSUEYT.png" width="400" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>Без функций активации нейронные сети могут выучивать только линейные зависимости. Чтобы аппроксимировать кривые, нам понадобятся функции активации.
</center></figcaption>
</figure>

**Функция активации** — это просто функция, которую мы применяем к каждому выходу слоя (его *активациям*). Самая распространённая — *rectifier* $max(0, x)$.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/aeIyAlF.png" width="400" alt="A graph of the rectifier function. The line y=x when x>0 and y=0 when x<0, making a 'hinge' shape like '_/'.">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

График rectifier-функции — это линия, у которой отрицательная часть «выпрямлена» в ноль. Применение функции к выходам нейрона даёт *изгиб* в данных, уводя нас от простых линий.

Когда мы присоединяем rectifier к линейному элементу, получаем **rectified linear unit** или **ReLU**. (По этой причине обычно и называют rectifier-функцию «ReLU-функцией».) Применение активации ReLU к линейному элементу означает, что выход становится `max(0, w * x + b)`, что можно изобразить так:

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/eFry7Yu.png" width="250" alt="Diagram of a single ReLU. Like a linear unit, but instead of a '+' symbol we now have a hinge '_/'. ">
<figcaption style="textalign: center; font-style: italic"><center>Выпрямленная линейная единица.
</center></figcaption>
</figure>

# Стекование слоёв Dense #

Теперь, когда у нас есть некоторая нелинейность, посмотрим, как можно стековать слои, чтобы получить сложные преобразования данных.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/Y5iwFQZ.png" width="450" alt="An input layer, two hidden layers, and a final linear layer.">
<figcaption style="textalign: center; font-style: italic"><center>Стек dense-слоёв образует «полносвязную» сеть.
</center></figcaption>
</figure>

Слои перед выходным слоем иногда называют **скрытыми**, поскольку мы никогда не видим их выходы напрямую.

Теперь обратите внимание, что последний (выходной) слой — линейный элемент (то есть без функции активации). Это делает сеть подходящей для задачи регрессии, где мы пытаемся предсказать произвольное числовое значение. Другие задачи (например, классификация) могут требовать функции активации на выходе.

## Построение моделей Sequential ##

Модель `Sequential`, которую мы использовали, соединяет список слоёв по порядку от первого к последнему: первый слой получает вход, последний слой выдаёт выход. Это создаёт модель, показанную на рисунке выше:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # the hidden ReLU layers
    layers.Dense(units=4, activation='relu', input_shape=[2]),
    layers.Dense(units=3, activation='relu'),
    # the linear output layer 
    layers.Dense(units=1),
])

Обязательно передавайте все слои вместе в списке, например `[layer, layer, layer, ...]`, а не отдельными аргументами. Чтобы добавить функции активации к слою, просто укажите её имя в аргументе `activation`.

# Ваш ход #

Теперь [**создайте глубокую нейронную сеть**](https://www.kaggle.com/kernels/fork/11887344) для набора данных *Concrete*.

---




*Есть вопросы или комментарии? Посетите [форум обсуждения курса](https://www.kaggle.com/learn/intro-to-deep-learning/discussion), чтобы пообщаться с другими учащимися.*